Converte arquivos em .PDF para texto puro .TXT

In [45]:
%%writefile HelperFunctions/TextMining/TextMining.py
            
# 1.Extração de texto dos pdf das sentenças baixas
#
# Cria Corpus (DatasetClassification.CSV) das Sentenças do TRT1
# Le os arquivos .txt e armazena-os em uma lista e depois persiste-o em disco no formato .csv.
#
# Ver notebook CriaCorpus.ipynb
#
# corpusTRT1 = Corpus()
# corpusTRT1.setDsFolder("\home\info\MyNotebooks\SentencasTRT1\Corpus")
# corpusTRT1.createTXT()  # Salva na subpasta \txt. Podemos criar um txt para cada arquivo .pdf ou um .txt gigante com o conteudo de todos os .pdf mas isso pode gerar problemas de memoria
# corpusTRT1.createCSV()  # Salva na subpasta \csv
# corpusTRT1.createDSDict() # Salva na subpasta \DS. Pega um .csv ou uma pasta com txt e cria o DatasetDict.
#
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import pathlib
import pandas as pd
from datasets import Dataset
from os import listdir
from os.path import isfile, join
import re
from datasets import load_dataset
    
class TM():
    
    def __init__(self):
        self.FOLDER_BASE = "/home/info/MyNotebooks/Datasets/SentencasTRT1/"
        self.PDFs_FOLDER = self.FOLDER_BASE + "PDFs/"
        self.TXTs_FOLDER = self.FOLDER_BASE + "TXTs/"
        self.CSV_FOLDER  = self.FOLDER_BASE + "CSV/"
        self.DS_FOLDER   = self.FOLDER_BASE + "DS/"
        self.ARQ_CSV     = "Corpus_Sentencas.csv"
        self.ARQ_DS      = "Corpus_Sentencas"
        
        
    # Dado um pdf especifico (PDF_FILE) le seu conteudo e retorna como texto
    def reading_pdf(self, PDF_FILE):
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        # detect_vertical=False, line_margin=0.4
        page_interpreter = PDFPageInterpreter(resource_manager, converter)

        with open(PDF_FILE, 'rb') as fh:

            #for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            for i, page in enumerate(PDFPage.get_pages(fh)):
                page_interpreter.process_page(page)

            text = fake_file_handle.getvalue()

        # close open handles
        converter.close()
        fake_file_handle.close()

        return(str(text))

    
    def getFileList(self):
        file_list = list()
        
        # Pega a lista de todos os arquivos na pasta PDFs_FOLDER
        files = [f for f in listdir(self.PDFs_FOLDER) if isfile(join(self.PDFs_FOLDER, f))]
        
        # Guarda na lista apenas aqueles arquivos que possuem extensão .PDF.
        for file in files:
            if pathlib.Path(file).suffix == ".pdf":
                file_list.append(file)
                
        return(file_list)

    
    def replacingExt(self, file_in):    
        # text = extract_text(join(folder_base, file))  # funcionou porem vem com assinatura digital na vertical
        arq = join(self.PDFs_FOLDER, file_in)
        print(arq)
        print(file_in)

        file_out = file_in
        file_out = file_out.replace(".pdf",".txt")
        file_out = join(self.TXTs_FOLDER, file_out)
        print(file_out)
        
        return(str(file_out))

    
    def removingSpeCarac(self, text_in):
        out = text_in.replace("\n", " ")
        out = out.replace("\t", " ")
        out = out.replace("\r", " ")
        out = out.replace("\f", " ")
        out = out.replace("\v", " ")
        out = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', out)
        
        return(str(out))

    
    def createTXT(self):
        # Processa os arquivos .pdf da pasta_base. Arquivos .txt servem também para o treinamento dos Tokenizadores
        
        files = self.getFileList()
     
        for file in files:
            
            arq = join(self.PDFs_FOLDER, file)
                
            text     = self.reading_pdf(arq)
            text     = self.removingSpeCarac(text)
            file_txt = self.replacingExt(file)

            with open(file_txt, 'w') as f:
                f.write(text)
            f.close()
                

Overwriting HelperFunctions/TextMining/TextMining.py


In [46]:
# 2. Criação do objeto
from HelperFunctions.TextMining.TextMining import TM
%run -i HelperFunctions/TextMining/TextMining.py
tm = TM()

In [47]:
%%time
# 3. Converção dos arquivos em .PDF para texto puro .TXT
tm.createTXT()

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01012895920185010432-DEJT-13-05-2019.pdf
01012895920185010432-DEJT-13-05-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01012895920185010432-DEJT-13-05-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01005865120175010081-DEJT-01-12-2017.pdf
01005865120175010081-DEJT-01-12-2017.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01005865120175010081-DEJT-01-12-2017.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003254520185010342-DEJT-06-06-2018.pdf
01003254520185010342-DEJT-06-06-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003254520185010342-DEJT-06-06-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01001941520185010522-DOERJ-03-04-2018.pdf
01001941520185010522-DOERJ-03-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01001941520185010522-DOERJ-03-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01013825720175010561-DEJT-08-12-2018.pdf
01013

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/00104983120155010050-DOERJ-26-07-2016.pdf
00104983120155010050-DOERJ-26-07-2016.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/00104983120155010050-DOERJ-26-07-2016.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/00106407920145010079-DEJT-06-08-2019.pdf
00106407920145010079-DEJT-06-08-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/00106407920145010079-DEJT-06-08-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01007224620175010017-DOERJ-18-04-2018.pdf
01007224620175010017-DOERJ-18-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01007224620175010017-DOERJ-18-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01006356020215010014-DEJT-27-10-2021.pdf
01006356020215010014-DEJT-27-10-2021.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01006356020215010014-DEJT-27-10-2021.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003224520185010066-DEJT-15-05-2018.pdf
01

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01001609820185010341-DEJT-13-03-2018.pdf
01001609820185010341-DEJT-13-03-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01001609820185010341-DEJT-13-03-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002450920185010075-DEJT-25-07-2018.pdf
01002450920185010075-DEJT-25-07-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002450920185010075-DEJT-25-07-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01008721020185010561-DEJT-28-05-2019.pdf
01008721020185010561-DEJT-28-05-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01008721020185010561-DEJT-28-05-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01001367020185010341-DEJT-22-03-2018.pdf
01001367020185010341-DEJT-22-03-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01001367020185010341-DEJT-22-03-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002255020185010226-DEJT-16-06-2018.pdf
01002255

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01005931120195010069-DEJT-02-08-2019.pdf
01005931120195010069-DEJT-02-08-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01005931120195010069-DEJT-02-08-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01001997720185010056-DEJT-15-05-2018.pdf
01001997720185010056-DEJT-15-05-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01001997720185010056-DEJT-15-05-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003960720185010226-DEJT-04-07-2018.pdf
01003960720185010226-DEJT-04-07-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003960720185010226-DEJT-04-07-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003822020185010421-DEJT-15-06-2018.pdf
01003822020185010421-DEJT-15-06-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003822020185010421-DEJT-15-06-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003246020185010342-DOERJ-10-05-2018.pdf
0100324

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01006946720215010040-DEJT-09-03-2022.pdf
01006946720215010040-DEJT-09-03-2022.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01006946720215010040-DEJT-09-03-2022.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002601020185010032-DEJT-14-04-2018.pdf
01002601020185010032-DEJT-14-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002601020185010032-DEJT-14-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002037720185010036-DEJT-03-04-2018.pdf
01002037720185010036-DEJT-03-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002037720185010036-DEJT-03-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003094720185010001-DEJT-25-04-2018.pdf
01003094720185010001-DEJT-25-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003094720185010001-DEJT-25-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003727320185010421-DEJT-05-11-2018.pdf
01003727

...

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01008026320205010030-DEJT-23-05-2022.pdf
01008026320205010030-DEJT-23-05-2022.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01008026320205010030-DEJT-23-05-2022.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01000144320195010011-DEJT-24-07-2019.pdf
01000144320195010011-DEJT-24-07-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01000144320195010011-DEJT-24-07-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002192820185010522-DOERJ-10-04-2018.pdf
01002192820185010522-DOERJ-10-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002192820185010522-DOERJ-10-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01000700320185010079-DOERJ-09-03-2018.pdf
01000700320185010079-DOERJ-09-03-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01000700320185010079-DOERJ-09-03-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002011320185010035-DEJT-27-11-2018.pdf
01

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01010567920165010061-DEJT-23-09-2017.pdf
01010567920165010061-DEJT-23-09-2017.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01010567920165010061-DEJT-23-09-2017.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002295620215010461-DEJT-04-08-2021.pdf
01002295620215010461-DEJT-04-08-2021.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002295620215010461-DEJT-04-08-2021.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01018707820175010248-DEJT-13-03-2019.pdf
01018707820175010248-DEJT-13-03-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01018707820175010248-DEJT-13-03-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01011127120185010246-DEJT-21-01-2019.pdf
01011127120185010246-DEJT-21-01-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01011127120185010246-DEJT-21-01-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002237820185010065-DEJT-30-07-2018.pdf
01002237

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01015338520175010411-DEJT-21-03-2019.pdf
01015338520175010411-DEJT-21-03-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01015338520175010411-DEJT-21-03-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003570720185010421-DEJT-27-06-2018.pdf
01003570720185010421-DEJT-27-06-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003570720185010421-DEJT-27-06-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01009794520175010058-DEJT-29-09-2018.pdf
01009794520175010058-DEJT-29-09-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01009794520175010058-DEJT-29-09-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/00112481620145010067-DOERJ-10-08-2017.pdf
00112481620145010067-DOERJ-10-08-2017.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/00112481620145010067-DOERJ-10-08-2017.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002133120185010066-DEJT-25-04-2018.pdf
01002

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01001692020185010031-DEJT-25-04-2018.pdf
01001692020185010031-DEJT-25-04-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01001692020185010031-DEJT-25-04-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01002739820185010261-DEJT-31-10-2018.pdf
01002739820185010261-DEJT-31-10-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01002739820185010261-DEJT-31-10-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01009088120165010283-DEJT-06-03-2018.pdf
01009088120165010283-DEJT-06-03-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01009088120165010283-DEJT-06-03-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01004093520195010302-DEJT-23-09-2021.pdf
01004093520195010302-DEJT-23-09-2021.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01004093520195010302-DEJT-23-09-2021.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01013996520175010053-DEJT-21-03-2019.pdf
01013996

/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01017885920175010244-DEJT-02-03-2021.pdf
01017885920175010244-DEJT-02-03-2021.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01017885920175010244-DEJT-02-03-2021.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01016775320185010531-DEJT-05-04-2019.pdf
01016775320185010531-DEJT-05-04-2019.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01016775320185010531-DEJT-05-04-2019.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01003325620185010077-DEJT-18-10-2018.pdf
01003325620185010077-DEJT-18-10-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01003325620185010077-DEJT-18-10-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01015616220175010020-DEJT-15-12-2018.pdf
01015616220175010020-DEJT-15-12-2018.pdf
/home/info/MyNotebooks/Datasets/SentencasTRT1/TXTs/01015616220175010020-DEJT-15-12-2018.txt
/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/01007271020195010046-DEJT-26-09-2019.pdf
01007271